In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from deduping_module import deduping_class

In [ ]:
link = r'C:\Users\USER\Documents\LM_project\SAL-230_Revisit validity'
gt = pd.read_csv(link + '\\raw\sf_export.csv', encoding='latin-1')
nm = pd.read_csv(link + '\\Texas\\texas-00001 - step 1.csv', encoding='latin-1')

#### Initializing the module

In [ ]:
# initializing the module
deduping = deduping_class(gt, 'account') 

In [ ]:
deduping.ground_truth.head()

#### Step 1:
1. Make sure that the ground truth columns are similar with the dataframe to be matched

In [ ]:
# matching the ground truth columns with the to match dataframe
nm.rename(columns={'firstName':'First Name', 'lastName':'Last Name', 'email':'Email', 'phone':'Phone', 'zip':'Zip Code 1', 'First and Last':'Account Name', 'street':'Street Address 1', 'state':'State 1'}, inplace= True)

In [ ]:
nm.head(3)

In [ ]:
nm.info()

In [ ]:
nm.info()

In [ ]:
nm.fillna('', inplace= True)

In [ ]:
nm[['Account Name', 'Street Address 1', 'State 1', 'Phone']]

#### Step 2
Now that the columns are matched, we can use the key_selector function. \
This takes 2 inputs, *args and "data=dataframe to be matched"\
This function will return the to be matched dataframe with primary_key value while also adding it in the ground truth but only in the backend

In [ ]:
deduping.key_selector('Account Name', 'Street Address 1', 'State 1', 'Phone', data= nm)

In [ ]:
# deduping.nm['State 1'] = deduping.nm.apply(lambda x: deduping.state_abbrev(x['State 1']), axis= 1)

#### Optional Step
We can set an optional paramater called ngrams, this means the number of combination the txt will be divided. \
if this is not set, it will automatically equal to 3

In [ ]:
deduping.set_ngrams(4)

#### Step 3
This function will convert the ground truth and dataframe to be matched in to tfidf sparse matrix \
the input to this function should be the dataframe with the primarykey \
the results will now be included in the class module

self.nm_tfidf\
self.gt_tfidf

In [ ]:
deduping.vectorizer()

In [ ]:
deduping.nm_tfidf_df

In [ ]:
deduping.gt_tfidf_df

#### Step 4
This function will get compare the similarities in the 2 dataframe and save the output in self.match

In [ ]:
deduping.get_match(1000000)

In [ ]:
deduping.matched

In [ ]:
deduping.non_matched_output

In [ ]:
deduping.matched_output

In [ ]:
# deduping.nm[~deduping.nm.index.isin(deduping.matched['index'].tolist())].to_csv('non_match_step 2.csv')

In [ ]:
# deduping.nm.merge(deduping.matched[['index','Ground Truth ID','similarity']].set_index('index'), left_index= True, right_index=True)

#### Notes:
1. If deduping at account object, always remember to remove the duplicates in the Salesforce Account Id
1. always make sure that the index of the inputs are in numerical order or this will cause errors in getting the matches
1. if a selected key value is missing, it has a significant impact on the performance

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
sns.boxplot(data=deduping.matched, x='similarity')

In [ ]:
sns.histplot(data=deduping.matched, x='similarity')

In [ ]:
deduping.matched.describe()

In [ ]:
deduping.matched.describe().loc['mean'][0]

#### Chebyshev's Theorem limits

In [ ]:
#Lower Limit
lower_limit = deduping.matched.describe().loc['mean'][0] - (2 * deduping.matched.describe().loc['std'][0])

In [ ]:

upper_limit = deduping.matched.describe().loc['mean'][0] + (2 * deduping.matched.describe().loc['std'][0])

In [ ]:
stopper

#### Performance Evaluation

In [ ]:
class deduping_performance:
    def __init__(self, nm, deduping_matched):
        self.df = nm.merge(deduping_matched[['index','Ground Truth ID','similarity']].set_index('index'), left_index= True, right_index=True)
        self.df.sort_values(by='similarity', ascending= False, inplace= True)
        self.df['similarity'] = self.df['similarity'].round(decimals= 3)
        self.df.drop_duplicates(subset=['similarity'], inplace =True)
        self.df.reset_index(drop=True, inplace= True)

        if len(self.df) < 25:
            raise ValueError('dataframe is less than threshold rows (25)')

        self.confirmation_list = [range(96,101), range(73,78), range(48,53),range(23,28),range(5)]
        self.get_sample()

    def get_percentile(self, percentile):
        percentile_value = self.df['similarity'].quantile((percentile/100), interpolation='lower')
        return self.df[self.df['similarity'] == percentile_value]

    def get_sample(self):
        temp_list= []
        for range in self.confirmation_list:
            for position in range:
                temp_list.append(self.get_percentile(position))
        self.confirmation_df = pd.concat(temp_list)
        self.confirmation_df.sort_values(by= 'similarity', ascending= False, inplace= True)

        

In [ ]:
test = deduping_performance(nm, deduping.matched)

In [ ]:
len(test.confirmation_df)

In [ ]:
test.confirmation_df.loc[(test.confirmation_df['similarity'] >= lower_limit) & (test.confirmation_df['similarity'] <= upper_limit), :]

In [ ]:
detected_duplicates = nm.merge(deduping.matched[['index','Ground Truth ID','similarity']].set_index('index'), left_index= True, right_index=True)

In [ ]:
len(detected_duplicates) - len(detected_duplicates[detected_duplicates['similarity'] == 1])

In [ ]:
len(detected_duplicates.drop_duplicates(subset=['Account Name'])) - len(detected_duplicates[detected_duplicates['similarity'] == 1].drop_duplicates(subset=['Account Name']))

In [ ]:
detected_duplicates.sort_values(by=['similarity'], ascending= False, inplace= True)

In [ ]:
test = deduping_performance(detected_duplicates)

In [ ]:
test.confirmation_df

In [ ]:
detected_duplicates.to_csv('Step 1 Matched_script.csv')

In [ ]:
Stopper

In [ ]:
import glob

demandtools_output = glob.glob(link +'\\raw\\*step 3*')
demandtools_output = [value for value in demandtools_output if 'Non_Match' not in value]

In [ ]:
temp_list = []
for file in demandtools_output:
    temp_df = pd.read_csv(file, low_memory=False)
    temp_list.append(temp_df)
demandtools_output = pd.concat(temp_list)

In [ ]:
demandtools_output.head(2)

In [ ]:
len(demandtools_output['First and Last'].unique())

In [ ]:
detected_duplicates[~detected_duplicates['Account Name'].isin(demandtools_output['First and Last'].unique().tolist())].drop_duplicates(subset=['Account Name'])

In [ ]:
demandtools_output[~demandtools_output['First and Last'].isin(detected_duplicates['Account Name'].unique().tolist())].drop_duplicates(subset=['First and Last'])

In [ ]:
test = detected_duplicates.merge(demandtools_output[['First and Last']].drop_duplicates(subset=['First and Last']), how='inner', left_on='Account Name', right_on='First and Last')